<a href="https://colab.research.google.com/github/tomasbsm/Modelo-Recomendacao/blob/main/Day_07_Din%C3%A2mica_Tom%C3%A1s5140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Blibiotecas e Importando as bases de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

sns.set(rc={'figure.figsize':(11.7,8.27)}) # isso é pras figuras ficarem maiores

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_itens = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DNC_order_items_dataset.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DNC_order_reviews_dataset.csv')
df_orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DNC_orders_dataset.csv')
df_products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DNC_products_dataset.csv')

In [3]:
df_itens.head()

# order_id - Id do pedido de compra;
# order_item_id - Número de produtos em uma mesma compra;
# product_id - Id do produto;
# price - preço;

,Unnamed: 0,order_id,order_item_id,product_id,price
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900


In [4]:
df_reviews.head()

#review_id - Id do review
#order_id - Id do pedido de compra
#review_score - Resulado do review

,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [5]:
df_orders.head()

# order_id - Id do pedido de compra
# customer_id - Id do consumidor que realizou a compra
# order_status - Status de Compra

,Unnamed: 0,order_id,customer_id,order_status
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [6]:
df_products.head()

#product_id - Id do produto
#product_category_name - Categoria do produto
#product_name_length - Tamanho do nome do produto
#product_description_length - Tamanho do nome do produto
#product_weigth_g - Peso do produto (em gramas)
#product_lenght_cm - Comprimento do produto (em cm)
#product_width_cm - Largura do produto (em cm)

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000,287.000,225.000,16.000,10.000,14.000
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000,276.000,1000.000,30.000,18.000,20.000
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000,250.000,154.000,18.000,9.000,15.000
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000,261.000,371.000,26.000,4.000,26.000
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000,402.000,625.000,20.000,17.000,13.000


#2. Definição do modelo
Com a base de dados disponibilizada, quais modeos de recomendação podem ser feitos?

**OFERECER O PRODUTO CERTO PARA A PESSOA CERTA**

**Qual é o melhor momento para rodar o modelo de recomendação?**

Em propagandas?

Quando o cliente estiver fazendo a pesquisa? 

Quando o cliente estiver vendo um produto?


## 2.1. Insights

1- Para oferecer o produto certo, precisa-se saber quais são os produtos que se parecem um com o outro (matriz de simaliridade). Para recomendar o produto para a pessoa certa, precisa-se saber qual o tipo de cliente com que ele se parece (matriz de simalaridade).

2- Além disso, esse modelo deve rodar enquanto o cliente estiver vendo um produto, pois nosso modelo de recomendação irá se basear em produtos similares!

#3. Exploração dos Dados
O que os dados nos dizem? Quais são as hipóteses que podem ser levantadas disso? Será que os dados são suficientes para o modelo que se propôs a fazer?

In [7]:
df_itens_original = df_itens.copy()
df_reviews_original = df_reviews.copy()
df_products_original = df_products.copy()
df_orders_original = df_orders.copy()

In [8]:
df_itens.shape

(112650, 5)

In [9]:
df_reviews.shape

(99224, 4)

In [10]:
df_products.shape

(32951, 9)

In [11]:
df_orders.shape

(99441, 4)

##3.1. Dataset de Reviews

In [12]:
df_reviews.info() #Não há dados nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    99224 non-null  int64 
 1   review_id     99224 non-null  object
 2   order_id      99224 non-null  object
 3   review_score  99224 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.0+ MB


In [13]:
df_reviews.drop(columns={'Unnamed: 0'}, inplace=True)
df_reviews.head()

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [14]:
df_reviews['review_score'].value_counts() #Imensa maioria das reviews são positivas

5    57328
4    19142
1    11424
3     8179
2     3151
Name: review_score, dtype: int64

##3.2. Dataset Itens

In [15]:
df_itens.describe() #Desvio padrão muito alto, média bem diferente da mediana (valores se concentram em preços mais baixos)

,Unnamed: 0,order_item_id,price
count,112650.000,112650.000,112650.000
mean,56324.500,1.198,120.654
std,32519.398,0.705,183.634
min,0.000,1.000,0.850
25%,28162.250,1.000,39.900
50%,56324.500,1.000,74.990
75%,84486.750,1.000,134.900
max,112649.000,21.000,6735.000


In [16]:
df_itens.drop(columns={'Unnamed: 0'}, inplace=True)
df_itens.head()

,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900


In [17]:
df_itens = df_itens.groupby(by=['order_id', 'product_id'], as_index=False) \
  .agg({'price': np.sum, 'order_item_id':np.max})
df_itens.head()

,order_id,product_id,price,order_item_id
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.900,1
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.900,1
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.000,1
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.990,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.900,1


In [18]:
df_itens.shape

(102425, 4)

In [19]:
# Verificando os produtos mais vendidos

df_top_products = df_itens.groupby(by='product_id', as_index=False) \
  .agg({'order_item_id': np.sum}) \
  .rename(columns={'order_item_id':'qtd'}) \
  .set_index('product_id') \
  .sort_values('qtd', ascending = False)

##3.3. Dataset Orders

In [20]:
df_orders.drop(columns={'Unnamed: 0'}, inplace=True)
df_orders['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [21]:
colunas_removidas = ["canceled", "unavailable", "invoiced", "processing", "created", "approved"]

In [22]:
df_orders['order_status']=df_orders['order_status'].apply(lambda x: 1 if x=="delivered" or x=="shipped" else 0)
df_orders['order_status'].value_counts()

1    97585
0     1856
Name: order_status, dtype: int64

##3.4. Dataset Products

In [23]:
df_products.drop(columns={'Unnamed: 0'}, inplace=True)
df_products.describe()

,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000,32341.000,32949.000,32949.000,32949.000,32949.000
mean,48.477,771.495,2276.472,30.815,16.938,23.197
std,10.246,635.115,4282.039,16.914,13.638,12.079
min,5.000,4.000,0.000,7.000,2.000,6.000
25%,42.000,339.000,300.000,18.000,8.000,15.000
50%,51.000,595.000,700.000,25.000,13.000,20.000
75%,57.000,972.000,1900.000,38.000,21.000,30.000
max,76.000,3992.000,40425.000,105.000,105.000,118.000


In [24]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_weight_g            32949 non-null  float64
 5   product_length_cm           32949 non-null  float64
 6   product_height_cm           32949 non-null  float64
 7   product_width_cm            32949 non-null  float64
dtypes: float64(6), object(2)
memory usage: 2.0+ MB


In [25]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000,287.000,225.000,16.000,10.000,14.000
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000,276.000,1000.000,30.000,18.000,20.000
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000,250.000,154.000,18.000,9.000,15.000
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000,261.000,371.000,26.000,4.000,26.000
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000,402.000,625.000,20.000,17.000,13.000


#4. Estruturação dos dados
Quais devem ser os dados de Treino? E os de Teste? Preciso juntar as informações entre as tabelas?

##4.1. Unificando as tabelas

In [26]:
df = df_itens.merge(df_orders, how='inner', on='order_id')
df.shape

(102425, 6)

In [27]:
df = df.merge(df_reviews, how='inner', on='order_id')
df.shape

(102230, 8)

In [28]:
df = df.merge(df_products, how='inner', on='product_id')
df.shape

(102230, 15)

In [29]:
df_merge = df.copy()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102230 entries, 0 to 102229
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    102230 non-null  object 
 1   product_id                  102230 non-null  object 
 2   price                       102230 non-null  float64
 3   order_item_id               102230 non-null  int64  
 4   customer_id                 102230 non-null  object 
 5   order_status                102230 non-null  int64  
 6   review_id                   102230 non-null  object 
 7   review_score                102230 non-null  int64  
 8   product_category_name       100773 non-null  object 
 9   product_name_lenght         100773 non-null  float64
 10  product_description_lenght  100773 non-null  float64
 11  product_weight_g            102214 non-null  float64
 12  product_length_cm           102214 non-null  float64
 13  product_height

In [31]:
df.head(10)

,order_id,product_id,price,order_item_id,customer_id,order_status,review_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.900,1,3ce436f183e68e07877b285a838db11a,1,97ca439bc427b48bc1cd7177abe71365,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.900,1,e6eecc5a77de221464d1c4eaff0a9b64,1,b11cba360bbe71410c291b764753d37f,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.900,1,4ef55bf80f711b372afebcb7c715344a,1,af01c4017c5ab46df6cc810e069e654a,4,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.900,1,30407a72ad8b3f4df4d15369126b20c9,1,8304ff37d8b16b57086fa283fe0c44f8,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.900,1,91a792fef70ecd8cc69d3c7feb3d12da,1,426f43a82185969503fb3c86241a9535,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
5,a0f9acf0b6294ed8561e32cde1a966bc,4244733e06e7ecb4970a6e2683c13e61,55.900,1,4851b3bca4a48b395dfad7aa57c3f72a,1,c752337ac465a5e2fbdbedf7d79c6659,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
6,bbf796534aaf9c59f8da8c7982db56e0,4244733e06e7ecb4970a6e2683c13e61,64.900,1,c32c207d9ebf75958011557ac1624e2f,1,b41646a4365a1411101396c097413907,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
7,c9ef97d2854afe64a3b4488bc2836af6,4244733e06e7ecb4970a6e2683c13e61,58.900,2,0a29abf475fb8039d2775913d6f0b6f0,1,dbacd3ac90ace7ad49fe8b31bbad2808,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
8,df2d910b8b5e5f461f67043489f9569d,4244733e06e7ecb4970a6e2683c13e61,55.900,1,2e76c1f2a920153d79cc98b634c50942,1,4e58c72bb0373240b2bd9bf84b493a69,1,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
9,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.900,1,f6dd3ec061db4e3987629fe6b26e5cce,1,7b07bacd811c4117b742569b04ce3580,4,pet_shop,56.000,239.000,30000.000,50.000,30.000,40.000


#5.Tratamento
Como pode ser feita a seleção e filtragem dos dados? Quais tratamentos devem ser feitos considerando os dados existentes?

In [32]:
df['order_status'].replace(0, np.nan, inplace=True)
df.dropna(subset=['order_status'], inplace=True)
df.shape

(101146, 15)

In [33]:
df.drop(columns='order_status', inplace=True)
df.head()

,order_id,product_id,price,order_item_id,customer_id,review_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.900,1,3ce436f183e68e07877b285a838db11a,97ca439bc427b48bc1cd7177abe71365,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.900,1,e6eecc5a77de221464d1c4eaff0a9b64,b11cba360bbe71410c291b764753d37f,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.900,1,4ef55bf80f711b372afebcb7c715344a,af01c4017c5ab46df6cc810e069e654a,4,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.900,1,30407a72ad8b3f4df4d15369126b20c9,8304ff37d8b16b57086fa283fe0c44f8,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.900,1,91a792fef70ecd8cc69d3c7feb3d12da,426f43a82185969503fb3c86241a9535,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000


#6. Criação do Modelo
Desenvolva o modelo!

In [34]:
df_Tratado = df.copy()

In [35]:
df_Tratado.shape

(101146, 14)

#6.1. Código para pegar os itens mais vendidos

In [36]:
df.head()

,order_id,product_id,price,order_item_id,customer_id,review_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.900,1,3ce436f183e68e07877b285a838db11a,97ca439bc427b48bc1cd7177abe71365,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.900,1,e6eecc5a77de221464d1c4eaff0a9b64,b11cba360bbe71410c291b764753d37f,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.900,1,4ef55bf80f711b372afebcb7c715344a,af01c4017c5ab46df6cc810e069e654a,4,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.900,1,30407a72ad8b3f4df4d15369126b20c9,8304ff37d8b16b57086fa283fe0c44f8,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.900,1,91a792fef70ecd8cc69d3c7feb3d12da,426f43a82185969503fb3c86241a9535,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000


In [37]:
top_itens_quantidade = df.groupby(by='product_id', as_index=False) \
  .agg({'order_item_id':np.sum, 'price':np.sum}) \
  .rename(columns = {'order_item_id':'quantidade vendida', 'price':'preço total'}) \
  .set_index('product_id') \
  .sort_values('quantidade vendida', ascending=False)
top_itens_quantidade.head(10)

,quantidade vendida,preço total
product_id,,
aca2eb7d00ea1a7b8ebd4e68314663af,526,37389.000
422879e10f46682990de24d770e7f83d,515,26680.120
99a4788cb24856965c36a24e339b6058,513,42306.360
389d119b48cf3043d311335e499d9c6b,419,21336.790
368c6c730842d78016ad823897a372db,407,21072.800
53759a2ecddad2bb87a079a1f1519f73,403,20391.200
d1c427060a0f73f6b889a5c7c61f2ac4,344,46618.510
53b36df67ebb7c41585e8d54d6772e08,325,37312.740
154e7e31ebfa092203795c972e5804a6,294,6573.080


In [38]:
top_itens_receita = df.groupby(by='product_id', as_index=False) \
  .agg({'order_item_id':np.sum, 'price':np.sum}) \
  .rename(columns = {'order_item_id':'quantidade vendida', 'price':'preço total'}) \
  .set_index('product_id') \
  .sort_values('preço total', ascending=False)
top_itens_receita.head(10)

,quantidade vendida,preço total
product_id,,
bb50f2e236e5eea0100680137654686c,196,64210.000
6cdd53843498f92890544667809f1595,156,54380.300
d6160fb7873f184099d9bc95e30376af,34,47249.350
d1c427060a0f73f6b889a5c7c61f2ac4,344,46618.510
99a4788cb24856965c36a24e339b6058,513,42306.360
3dd2a17168ec895c781a9191c1e95ad7,273,40782.800
25c38557cf793876c5abdd5931f922db,38,37908.320
5f504b3a1c75b73d6151be81eb05bdc9,64,37733.900
aca2eb7d00ea1a7b8ebd4e68314663af,526,37389.000


##6.2. Clientes que mais compraram

In [39]:
df.head()

,order_id,product_id,price,order_item_id,customer_id,review_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.900,1,3ce436f183e68e07877b285a838db11a,97ca439bc427b48bc1cd7177abe71365,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.900,1,e6eecc5a77de221464d1c4eaff0a9b64,b11cba360bbe71410c291b764753d37f,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.900,1,4ef55bf80f711b372afebcb7c715344a,af01c4017c5ab46df6cc810e069e654a,4,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.900,1,30407a72ad8b3f4df4d15369126b20c9,8304ff37d8b16b57086fa283fe0c44f8,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.900,1,91a792fef70ecd8cc69d3c7feb3d12da,426f43a82185969503fb3c86241a9535,5,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000


In [40]:
top_clientes_quantidade = df.groupby(by='customer_id', as_index=False) \
  .agg({'order_item_id':np.sum, 'price':np.sum}) \
  .rename(columns = {'order_item_id':'quantidade vendida', 'price':'preço total'}) \
  .set_index('customer_id') \
  .sort_values('quantidade vendida', ascending=False)
top_clientes_quantidade.head(10)

,quantidade vendida,preço total
customer_id,,
be1c4e52bb71e0c54b11a26b8e8d59f2,82,1430.160
fc3d1daec319d62d49bfb5e1f83123e9,52,31.800
0d861a5e4dd6a9079d89e1330848f0ab,43,1138.300
daf15f1b940cc6a72ba558f093dc00dd,31,958.800
a67a246af6ba598a14cc86df3c0354ee,28,563.000
30bb84b541c96af98ba7d90b9ebf35d0,28,1093.000
1205480caca6c37f55954da838933b8a,28,186.590
50920f8cd0681fd86ebe93670c8fe52e,26,306.770
679f84ceb2ee4ca5bca0c3ea34647746,26,479.200


##6.3. Funções base para cálculo de similaridade

In [41]:
# Faz uma cópia para que o dataset original não seja alterado
def start_pipeline(df):
    return df.copy()

# Agrupa o dateset por usuários e conta quantos livros diferentes cada um tem
# Ordena para que os usuários com mais livros fiquem no topo do dataset
def get_user_counts(df): 
  return df.groupby(by='customer_id', as_index=False) \
    .agg({'product_id': pd.Series.nunique}) \
    .rename(columns={'product_id': 'product_id_count'}) \
    .set_index('customer_id') \
    .sort_values('product_id_count', ascending = False)

# Agrupa o dateset por livros e conta quantos usuários diferentes cada lirvro tem
# Ordena para que os livros com mais usuários diferentes fiquem no topo do dataset
def get_product_counts(df): 
  return df.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_product_review(df): 
  return df.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review'}) \
    .set_index('product_id') \
    .sort_values('review', ascending = False)

#Recebe o dataset e o tamanho da amostra
 # Retorna as n primeiras linhas do dataset recebido em formato de lista
def get_n_top_values(df, n): 
  return df.head(sample_size) \
    .index \
    .tolist()

 # Executa as funções acima para rankear os usuários
sample_size = 100
top_user_ids = df \
  .pipe(start_pipeline) \
  .pipe(get_user_counts) \
  .pipe(get_n_top_values, sample_size)

# Executa as funções acima para rankear os produtos mais vendidos
sample_size = 99
top_product_ids = df \
  .pipe(start_pipeline) \
  .pipe(get_product_counts) \
  .pipe(get_n_top_values, sample_size)

# Executa as funções acima para rankear os produtos mais bem avaliados
sample_size = 97
top_product_review = df \
  .pipe(start_pipeline) \
  .pipe(get_product_review) \
  .pipe(get_n_top_values, sample_size)

###6.3.1. Matriz de similaridade para clientes

In [42]:
#Transforma a amostra em um dataset, renomeando as colunas.
def transform_sample(df):
  return pd.DataFrame({
      "user": df["customer_id"],
      "product": df["product_id"],
      "purchase": df["order_item_id"]
  })

# Seleciona só os User-IDs que estão entre os top Users, transforma amostra em um dataframe e organiza o dataset em uma matriz Cliente x Produto 
matriz_de_relacionamento_user_sampled = df[df["customer_id"].isin(top_user_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="product", values="purchase").fillna(0)

In [43]:
matriz_de_relacionamento_user_sampled

product,0011c512eb256aa0dbbb544d8dffcf6e,0111ab6fb41da433e0f17d3f4ec2c8f5,01c2e91674406ebaca6a1bbf7f61c3f0,01d4157fd9738ea81038316ae4cfc192,0333795efd58e3c33c5850b92721c3e4,03d817e5e392e78674ed0bd8195f9159,0522015abfd1094949180dee8dacbdf9,05805f52cdeb93d7684f871e599f0078,060ea530ba6184d49f7b1cc031c4c97e,060f0122cdc8f25db4a0d7359fdae7b0,064c975d37d3c646bd94d94c6e16f37e,08ef02b6aae73ad3a385cf1342f609c7,0a4093a4af429dc0a9334300e5c13ae5,0a56efd5f050d3f861a04c6f005d1128,0a9a9c25c5a06532d1766e00db833a7f,0bcc3eeca39e1064258aa1e932269894,0bdd9a52d192d64770e33ae559512478,0cf13ac73dbcf6586ba63b89dd1f780a,0cf2faf9749f53924cea652a09d8e327,0dce4d890959f0607b58e7475d13fc86,0de59eddc63167215c972b0d785ffa7b,0e2c6b2fc22b192f824a68a0fa537aa0,0e35a413d832e63c5d46101b7c88de0f,0e92dcf7b03df1de4e9349af065f8e63,0f6e214a2c5baab8c1ccb053a0829f13,0fa81e7123fd0ebe03adbbe99d912827,1065e0ebef073787a7bf691924c60eeb,113f80f12c8892f0c59206f70b862b40,1179867d805886591809f49767db9ecd,12ad279319eff927d9333b42f3382ff8,154696c4f8e96ed48f0c191eb09c0683,155815f7173905f1e5cde0b82adfc113,15b5d9fcf84c060e00b0883def7d10e6,17d8b8e9c21ec0fc5b33724e32fe422e,186fe07c8b8005ec6a498587ffbc1352,18796df281656da4036dd926561a6030,18fa9cc25ea8b54f32d029f261673c0f,1a06a6a66ab23d70e02b8f92650e268f,1aa2ed670a95e9f475723707a7c3f386,1b0e9a414e59a00b13262ffb248ed9b4,...,e256d05115f9eb3766f3ab752132a4e2,e273c62ea30d5ca1753739e6d25d15a1,e478d0bd1efbc9329837d30c174bb850,e4c7ed7a832858c17d60ad5e38b00aba,e5897f494af72d083f3899808f341659,e6e3140cdd035e915d6d4a9eae2ccead,e76572c2ae4cede1e3b2ead512423efe,e82b2802625252b6d72b3e1d36f82670,e892e0e01990e737eb81a57db12f8c8c,e95fb3a80ca294f39de925b6428f3a32,e9e1e0a297e5a8f045e8c0395c4c745b,eaf84e31d1e891449aa5cf7130e75fc6,eb048e296b8db6cd64b1bafd9500786c,eb53f94fdc60278efcef123bb275658a,ebf969336461d98e41d31673d8af75ba,ecfb5255f55715e1faec3e6a316da6c3,ede062bcd13ef64b5cc87369e4edaf62,eeab31cd2a4bcd64e6b249e0a674b3a8,ef66e63137bdd1b2e44aa968646fc1e3,f16e5b7883c1349e57bc5e92ba9e0d96,f2bec64d5d795250be5e5ec2d38a19b6,f2d649f157cebee085ed738f23d5d6b7,f2e53dd1670f3c376518263b3f71424d,f303e2cdf0967ef1a153d12f2685d280,f35013cb05a9538a4428fa6161b2c96b,f3a0e8de0d8bada90f638dd837416ad7,f3a1a865ba61412ac17ab53f1a9c0082,f3fad6d734fb27bf8a548562e04f9a5e,f4d705aa95ccca448e5b0deb6e5290ba,f97194491a605b1d0e50d631e443cb5a,f97a136cf6e5f5d9333b513afc8b153e,fa243e761ae871597627e917611eeb23,fa6abc53b6abfed3231c37cfa290850a,fa814ebf85f1640b2acea3c4378d40f9,fb783e3e545937820b57fe539b2c5a6c,fbce4c4cb307679d89a3bf3d3bb353b9,fc6b299cf83fa76632dd1b0a2c611f33,fd440fd792fd8a4b681f1eb258c25445,fd471a043ee8b8dd27f4086495e0724c,fef0dd8c53552d0f784b7cc5375a3ffe
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00ca6e4277e535d41bda84d371310be2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
09cc193befa14d698869667e92148ab2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0a4a8ee3c788108918b12f21435c950c,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculando a similaridade entre os usuários através da multiplicação da matriz CxP pela sua transposta
cos_user = cosine_similarity(matriz_de_relacionamento_user_sampled)
#Imprimindo o resultado
cos_user.sum()

106.15011306983709

###6.3.2. Matriz de Similaridade de produto

In [45]:
matriz_de_relacionamento_product_sampled = df[df["product_id"].isin(top_product_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="product", values="purchase").fillna(0)

In [46]:
matriz_de_relacionamento_product_sampled.T.head(10)

user,00066ccbe787a588c52bd5ff404590e3,000e943451fc2788ca6ac98a682f2f49,000fd45d6fedae68fc6676036610f879,001028b78fd413e19704b3867c369d3a,0019c9aaad15b043c48f0a1180f22ce8,004f39eb181d53fa796fe82ce7a0cbd5,0054f2f885ca69abcf4935c9506d7851,00573ea7a2ac9dff51ac3d177cd6eaea,00620f738107803e6c0f6818a7a3b7e3,006496598c918064dc19eef95e5e47f8,00652f7a6e012d58f58fefd69d1a1ea4,006a5d6b5f648f3811fd4fa94d93a67c,00701ade29704fc9aa33a8766b082cb6,00765aebd8e20a9a8aa8c28d0099a938,0078867a1043bd408c6134cf5ca1bbf5,007a444502902587d7aa72af38cbd1de,007a6cde976f335c04a22d7043dd23f0,008943f202ed451804dfc1c23248eeb0,0093eadfb5536c158311672bdf418044,00949af96107ff1f55bc0f09e913686d,0098400373e087d3c231fb5d37834fde,009e30667771830117017cf8581b98a5,00a860aeb15fb205efdb0c689d8f9c6a,00acc750f0c2bda99b95c4043ff1e662,00af1761df71da362764c51bb96f5118,00b366712a38454d2e62c9d2509d0623,00b5072ef1192bc4d02e1d6264317bb0,00b694184c8c2f2a565e4def5a00b8ee,00ba24d5e8869fd25950fd26c098ee20,00c5b008c1465e327578fdacf185ccfa,00cef6d9683eaa39aaf401e3b02bdbca,00d4c7cec24152e73f88312fdbf09ffa,00d9314fb3f147133d0dae548abab307,00da45766aa7d360508cf25434334a8d,00e5b2926b61c618f32e26a8683d84c7,00e7c7983371f6e0c6ce890040d431db,00e7c822a5cd0647e9251fc711276ea7,00f3b3a7cd0b6566435090c7fbda03a2,00f8ac751ae484a82adb20b42ad1aca7,00fd2c64e6e3f94a81541196fc536f4f,...,ff28bc415971e058ac40d7dd4e4181e3,ff30b76fc9c812f006ee87f55fba0e93,ff319a4ce17236c6a104237b7970f479,ff321454b4ea8fda58a0d3a0ab2609cb,ff33ddf6fafd5d38aaa6c0f7385e418f,ff349ea42c06633992fc3c94c3da0f58,ff3aa1123b9a7d52d6043adec012e33f,ff3fb57f76210bacea765b1f7d8876ad,ff43182cf77e7ee5609976d3f7a7bade,ff44ec08cbec7536134c279fc52ae437,ff4b5d3b660b635bcbdb988ca60eafbe,ff53737545b992cfdf52fcfba37e2424,ff55a5afd93c0f7de549bd83e418f55a,ff55e4b1163819f09cb71434a8e72bff,ff5ca282b802097cc7182d5f4b58d199,ff6414bc057a2275b727e817428a1637,ff774d667d8f3f8fe0bdd578e8d0a85f,ff792fe1772fd36b73b4ef17a42f7e6c,ff83b1b9ffb0f2f60273867bb26f671e,ff869205a42a4de9aa4a4bc357c62507,ff886777efd58cd30781969d96dfe558,ff8b10ae20558c8201652e505e41b0fb,ff8f6d3e8686235229636f5b388e3b44,ff8fa0e333a5b7b5362e76311b4ae90a,ff9f5774bdf98c9eb203a168b58ded57,ffa219109978b03ab5d6340856861021,ffb0ffdb00e2e0b31c423577db3520eb,ffbc1b5517c488c7959e231ff9a0a787,ffbcf1eb664ce45bed8883fbcbdc2b06,ffbda9243760f2492df6e71c590e070a,ffc1df0bc187796047379f3b99a4f08c,ffc5e5000221495c2cbb2b099aaec969,ffcbcec4480d074ffd50398e66bff27c,ffd940dedc35e4f459eddfa41291e824,ffdaaad4323306777060ca43b3f87e7f,ffdd80d8f3655192a9d2a38d5b18732c,ffe1eab23bff108bf37c973b05d4e9ba,ffed5d4f6245fe13cd57054621eec36a,fff212062d600f2e1d53f3c5d4a25138,fffb97495f78be80e2759335275df2aa
product,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
06edb72f1e0c64b14c5b79353f7abea3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
08574b074924071f4e201e151b152b4e,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0a57f7d2c983bcf8188589a5fea4a8da,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

In [47]:
# Calculando a similaridade entre os livros através da multiplicação da matriz Produto x Cliente pela sua transposta 
cos_item = cosine_similarity(matriz_de_relacionamento_product_sampled.T)
# Imprimindo o resultado
print(cos_item)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [48]:
cos_item.sum()

103.23977176982498

##6.4. Testando o modelo

###6.4.1. Recomendação para produtos mais similares

In [52]:
corr_items = np.corrcoef(matriz_de_relacionamento_product_sampled.T)

produto_de_interesse = matriz_de_relacionamento_product_sampled.T.index[-1]

print(f"Produto de interesse: {produto_de_interesse}")

produto_idx = matriz_de_relacionamento_product_sampled.T.index.tolist().index(produto_de_interesse)
print(f"Produto de id: {produto_de_interesse},tem índice {produto_idx}")

closest_10_products = np.argsort(-corr_items[produto_idx])[:10]

for i in zip(matriz_de_relacionamento_product_sampled.T.index[closest_10_products], corr_items[produto_idx][closest_10_products]):
    print(f"Produto {i[0]} tem similaridade {i[1]:.5f} com produto {produto_de_interesse}")

Produto de interesse: fb55982be901439613a95940feefd9ee
Produto de id: fb55982be901439613a95940feefd9ee,tem índice 98
Produto fb55982be901439613a95940feefd9ee tem similaridade 1.00000 com produto fb55982be901439613a95940feefd9ee
Produto dbb67791e405873b259e4656bf971246 tem similaridade -0.00440 com produto fb55982be901439613a95940feefd9ee
Produto 5a6e53c3b4e8684b13388d6aa4afdf12 tem similaridade -0.00497 com produto fb55982be901439613a95940feefd9ee
Produto cec09725da5ed01471d9a505e7389d37 tem similaridade -0.00526 com produto fb55982be901439613a95940feefd9ee
Produto 93c902b021a9e594f658ab1b0351602a tem similaridade -0.00531 com produto fb55982be901439613a95940feefd9ee
Produto 43423cdffde7fda63d0414ed38c11a73 tem similaridade -0.00532 com produto fb55982be901439613a95940feefd9ee
Produto 2948658cb6abc82847412be7201bfc4c tem similaridade -0.00533 com produto fb55982be901439613a95940feefd9ee
Produto ab1f9387c0627dd24000bfbc54fdeee0 tem similaridade -0.00540 com produto fb55982be901439613a95

###6.4.2. Recomendação para clientes mais similares

In [51]:
corr_users = np.corrcoef(matriz_de_relacionamento_user_sampled)

user_de_interesse = matriz_de_relacionamento_user_sampled.index[-1]

print(f"User de interesse: {user_de_interesse}")

user_idx = matriz_de_relacionamento_user_sampled.index.tolist().index(user_de_interesse)
print(f"User de id: {user_de_interesse},tem índice {user_idx}")

closest_10_users = np.argsort(-corr_users[user_idx])[:10]

for i in zip(matriz_de_relacionamento_user_sampled.index[closest_10_users], corr_users[user_idx][closest_10_users]):
    print(f"User {i[0]} tem similaridade {i[1]:.5f} com user {user_de_interesse}")

User de interesse: fee8071ecc5ad2f5b2831c4fbd9b3cdd
User de id: fee8071ecc5ad2f5b2831c4fbd9b3cdd,tem índice 99
User fee8071ecc5ad2f5b2831c4fbd9b3cdd tem similaridade 1.00000 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 12560c8fc6ef4609768854c0dec45c80 tem similaridade -0.00675 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 837bffa34c99bc9aed29abf1b5bdd886 tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User e804e7ef5f81657fd973fde6a032d3f4 tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 6c185ed34983b0d80f77402efb2453bc tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 00ca6e4277e535d41bda84d371310be2 tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User a51ebdfce7cdd6930370083085f240c6 tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 2e70fb007f814f4e0bc167ca49d271f6 tem similaridade -0.00709 com user fee8071ecc5ad2f5b2831c4fbd9b3cdd
User 8c4a739a0897f0706505e419d7f1e4ac tem 

#7. Ajustes no Portfólio
O que pode ser melhorado para que o portfólio seja bem explicativo e estruturado?

Para melhorar o portfólio poderíamos colocar todo o desenvolvimento deste notebook em um arquivo .py para que fique mais fácil para o usuário utilizar o modelo de recomendação e visualizar o que está sendo feito. Além disso, a parte do código referente a análise exploratória não precisa obrigatoriamente fazer parte do modelo, pode ser inserida como um arquivo anexo justificando as escolhas feitas ao longo do nosso desenvolvimento.